In [146]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os
import matplotlib.pyplot as plt
import urllib.request


In [145]:
#categorized census tracts into 4 categories based on income level
def tract_type(row):
    if (row["tract_to_msamd_income"] < 50):
        return "low"
    if (row["tract_to_msamd_income"] < 80):
        return "mod"
    if (row["tract_to_msamd_income"] <= 120):
        return "mid"
    if (row["tract_to_msamd_income"] > 120):
        return "high"
    else:
        return "not categorized"
    


In [171]:
#codes a tract as CRA eligible or ineligible based on its median income to AMI ratio
def tract_eligibility(row):
    ratio = row["tract_to_msamd_income"]
    if (ratio < 80):
        return "eligible"
    else:
        return "not eligible"

In [ ]:
def tract_inc_ratio(row):
    return 

In [143]:
#codes a loan borrower as CRA eligible or ineligible based on their median income to AMI ratio
def borrower_eligibility(row):
    if (row["applicant_income_000s"]*1000 < .8*row["hud_median_family_income"]):
        return "eligible"
    else:
        return "not eligible"


In [142]:
def add_year(df, year):
    df["Year"] = year
    

In [141]:
#calculates the proportion of loans in the dataset that go to each of the 4 person-place categories, returns an array of 
#4 proportions that correspond to each of the 4 categories
def people_vs_place(df):
    hb_lt = 0 
    hb_ht = 0
    lb_lt = 0
    lb_ht = 0
    for row in df.itertuples():
        cra_eligible_tract = row._48 
        cra_eligible_borrower = row._49

        if (cra_eligible_tract == "eligible" and cra_eligible_borrower == "eligible"):
            lb_lt +=1
            
        elif (cra_eligible_tract == "not eligible" and cra_eligible_borrower == "eligible"):
            lb_ht +=1
        elif (cra_eligible_tract == "eligible" and cra_eligible_borrower == "not eligible"):
            hb_lt +=1
        elif (cra_eligible_tract == "not eligible" and cra_eligible_borrower == "not eligible"):
            hb_ht +=1
        
    return [hb_lt/len(df), hb_lt, hb_ht/len(df), hb_ht, lb_lt/len(df), lb_lt, lb_ht/len(df), lb_ht]
    
        

In [140]:
bay_county_names = ["Alameda", "ContraCosta", "Sonoma", "Solano", "SanMateo", "SantaClara", "SanFrancisco", "Marin","Napa"] 
#bay_county_codes = ['001', '013', '041', '055', '075', '081', '085', '097', '095']

#note- 075 is SF, since SF CTs are 3 digits long, I added a 0 to the county code so that the 10 digit FIPS would be 
#correct and consistent with the census
bay_county_codes = ['001', '013', '097', '095', '081', '085', '0750', '041', '055']

counties = {"Alameda": '001', "ContraCosta": '013', "Sonoma":  '097', "Solano": '095', "SanMateo": '081', 
            "SantaClara": '085', "SanFrancisco": '0750',  "Marin": '041', "Napa": '055'}

analysis_years = ["2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]



In [174]:
def parse_data(df, year, county):
#this uses the functions above combined to "parse" raw HMDA data, including to generate proportions of 
#loans to combinations of HI/LI borrowers and tracts and append other columns described above. It also adds a column 
#called geoid which is the full 10 digit FIPS for the census tract, which is useful for later processing.
    df["CRA eligible tract"] = df.apply(lambda item: tract_eligibility(item), axis=1)
    df["CRA eligible borrower"] = df.apply(lambda item: borrower_eligibility(item), axis=1)
    df["type"] = df.apply(lambda item: tract_type(item), axis=1)
    df["Year"] = year
    df["inc_ratio"] = df.apply(lambda item: item["tract_to_msamd_income"], axis=1)
    
    df_lol = []
    for tract, tract_df in df.groupby("census_tract_number"):
        row_list = [tract, 
                    tract_df["Year"].iloc[0], tract_df["county_name"].iloc[0],
                    tract_df["type"].iloc[0], tract_df["CRA eligible tract"].iloc[0], tract_df["inc_ratio"].iloc[0]]
        row_list.extend(people_vs_place(tract_df))
        df_lol.append(row_list)
    data = pd.DataFrame(df_lol, columns=["Tract","Year", "County", "type", "CRA Eligible", "inc_ratio", '% HI borrower, LI tract', '# HI borrower, LI tract', 
                                         '% HI borrower, HI tract','# HI borrower, HI tract', "% LI borrower, LI tract", "# LI borrower, LI tract",
                                         "%LI borrower, HI tract", "# LI borrower, HI tract"])
    return data
        
        

In [180]:


#It takes several minutes to run since it goes through all of the years and counties of data. The end result is a
#csv file for each year and county with a line for each tract and information about the type of tract, the CRA eligibiliy
#and the number and percentage of each borrower-tract combination
filenames = []
tables_dict = {}
for county, code in zip(bay_county_names, bay_county_codes): 
    for year in analysis_years:
        filename = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/raw_hmda_filtered/"+ county + "_"+ str(year)+ ".csv"
        filenames.append(filename)
        df = pd.read_csv(filename)
        new_frame = parse_data(df, year, county)
        new_frame["Geoid"] = '6'+code+((new_frame["Tract"]*100).astype(int)).astype(str)

        tables_dict[county+ "_"+ str(year)] = new_frame
        print("writing..", county, year, " to csv")
        #WRITING NEW DATA TO NEW FOLDER AS TEST
        new_frame.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/parsed_data_1/"+county+"_"+str(year)+"_parsed.csv", index=False)
              

writing.. Alameda 2008  to csv
writing.. Alameda 2009  to csv
writing.. Alameda 2010  to csv
writing.. Alameda 2011  to csv
writing.. Alameda 2012  to csv
writing.. Alameda 2013  to csv
writing.. Alameda 2014  to csv
writing.. Alameda 2015  to csv
writing.. Alameda 2016  to csv
writing.. Alameda 2017  to csv
writing.. ContraCosta 2008  to csv
writing.. ContraCosta 2009  to csv
writing.. ContraCosta 2010  to csv
writing.. ContraCosta 2011  to csv
writing.. ContraCosta 2012  to csv
writing.. ContraCosta 2013  to csv
writing.. ContraCosta 2014  to csv
writing.. ContraCosta 2015  to csv
writing.. ContraCosta 2016  to csv
writing.. ContraCosta 2017  to csv
writing.. Sonoma 2008  to csv
writing.. Sonoma 2009  to csv
writing.. Sonoma 2010  to csv
writing.. Sonoma 2011  to csv
writing.. Sonoma 2012  to csv
writing.. Sonoma 2013  to csv
writing.. Sonoma 2014  to csv
writing.. Sonoma 2015  to csv
writing.. Sonoma 2016  to csv
writing.. Sonoma 2017  to csv
writing.. Solano 2008  to csv
writing.. 

In [194]:
new_frame.dtypes

Tract                      float64
Year                        object
County                      object
type                        object
CRA Eligible                object
inc_ratio                  float64
% HI borrower, LI tract    float64
# HI borrower, LI tract      int64
% HI borrower, HI tract    float64
# HI borrower, HI tract      int64
% LI borrower, LI tract    float64
# LI borrower, LI tract      int64
%LI borrower, HI tract     float64
# LI borrower, HI tract      int64
Geoid                       object
dtype: object

### Create master CT reference by Type

In [195]:
#create a master reference dataframe that maps all census tracts in the bay to their type. This uses information 
#directly from the HMDA data, specifically the "tract to median income"
path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/parsed_data_1/"
files = [f for f in os.listdir(path) if f[-3:] == "csv"]
dfs = []
for file in files:
#     print(file)
#     print(len(files))
    
    df = pd.read_csv(path+file)
    lite = df[["Geoid", "Tract", "Year", "type", "County", "CRA Eligible", "inc_ratio", "# HI borrower, LI tract",
        "# HI borrower, HI tract", "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(lite)
    
master = pd.concat(dfs, axis=0)
len(master)
master[master["Year"] == 2009]['County'].unique()


array(['Contra Costa County', 'Sonoma County', 'Marin County',
       'San Mateo County', 'Napa County', 'San Francisco County',
       'Santa Clara County', 'Alameda County', 'Solano County'],
      dtype=object)

In [196]:
master.head()

,Geoid,Tract,Year,type,County,CRA Eligible,inc_ratio,"# HI borrower, LI tract","# HI borrower, HI tract","# LI borrower, LI tract","# LI borrower, HI tract"
0,6013301000,3010.00,2015,mid,Contra Costa County,not eligible,82.510002,0,82,0,14
1,6013302005,3020.05,2015,mid,Contra Costa County,not eligible,85.599998,0,35,0,30
2,6013302006,3020.06,2015,mid,Contra Costa County,not eligible,95.730003,0,21,0,24
3,6013302007,3020.07,2015,mid,Contra Costa County,not eligible,81.199997,0,59,0,26
4,6013302008,3020.08,2015,mid,Contra Costa County,not eligible,83.910004,0,74,0,21


In [197]:
master.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/reference/tracts_type_master_1.csv", index=0) #exports it to csv

#### Get list of tracts with their income ratios for each year (for visualization)

In [12]:
test = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/raw_hmda_filtered/Alameda_2008.csv")
print(test.columns)
test.head()

Index(['tract_to_msamd_income', 'rate_spread', 'population',
       'minority_population', 'number_of_owner_occupied_units',
       'number_of_1_to_4_family_units', 'loan_amount_000s',
       'hud_median_family_income', 'applicant_income_000s', 'state_name',
       'state_abbr', 'sequence_number', 'respondent_id', 'purchaser_type_name',
       'property_type_name', 'preapproval_name', 'owner_occupancy_name',
       'msamd_name', 'loan_type_name', 'loan_purpose_name', 'lien_status_name',
       'hoepa_status_name', 'edit_status_name', 'denial_reason_name_3',
       'denial_reason_name_2', 'denial_reason_name_1', 'county_name',
       'co_applicant_sex_name', 'co_applicant_race_name_5',
       'co_applicant_race_name_4', 'co_applicant_race_name_3',
       'co_applicant_race_name_2', 'co_applicant_race_name_1',
       'co_applicant_ethnicity_name', 'census_tract_number', 'as_of_year',
       'application_date_indicator', 'applicant_sex_name',
       'applicant_race_name_5', 'applicant_rac

,tract_to_msamd_income,rate_spread,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,state_name,...,applicant_sex_name,applicant_race_name_5,applicant_race_name_4,applicant_race_name_3,applicant_race_name_2,applicant_race_name_1,applicant_ethnicity_name,agency_name,agency_abbr,action_taken_name
0,54.070000,NaN,4451.0,83.099998,718.0,1688.0,252,86100.0,68.0,California,...,Male,NaN,NaN,NaN,NaN,White,Not Hispanic or Latino,Office of the Comptroller of the Currency,OCC,Loan originated
1,59.990002,NaN,1252.0,38.740002,9.0,443.0,622,86100.0,242.0,California,...,Female,NaN,NaN,NaN,NaN,White,Not Hispanic or Latino,Office of the Comptroller of the Currency,OCC,Loan originated
2,109.290001,NaN,2877.0,28.780001,837.0,987.0,412,86100.0,135.0,California,...,Male,NaN,NaN,NaN,NaN,Asian,Not Hispanic or Latino,Office of the Comptroller of the Currency,OCC,Loan originated
3,84.930000,NaN,4715.0,41.910000,771.0,1589.0,417,86100.0,176.0,California,...,Male,NaN,NaN,NaN,NaN,White,Not Hispanic or Latino,Office of the Comptroller of the Currency,OCC,Loan originated
4,77.980003,NaN,3410.0,62.820000,586.0,1179.0,344,86100.0,107.0,California,...,Male,NaN,NaN,NaN,NaN,White,Not Hispanic or Latino,Office of the Comptroller of the Currency,OCC,Loan originated


In [9]:
test[["tract_to_msamd_income", "census_tract_number"]]

,tract_to_msamd_income,census_tract_number
0,54.070000,4007.00
1,59.990002,4274.00
2,109.290001,4427.00
3,84.930000,4278.00
4,77.980003,4005.00
5,168.600006,4211.00
6,34.470001,4016.00
7,103.879997,4417.00
8,96.320000,4004.00
9,131.210007,4099.00


In [134]:
def get_full_fips(tract, county):
    try:
        county_code = counties[county]
        print("county code", county_code)
        return "6"+county_code+str(int(tract*100))
    except Exception as e:
        print(tract, e)
        return float('nan')

In [135]:
#create a master reference dataframe that maps all census tracts in the bay to their type. This uses information 
#directly from the HMDA data, specifically the "tract to median income"
path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/raw_hmda_filtered/"
reference = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/reference/'

filenames = []
year_dfs = []
tables_dict = {}

for year in ["2009", "2013", "2014", "2015", "2016", "2017"]:
    for county in counties: 
        filename = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/raw_hmda_filtered/"+ county + "_"+ year+ ".csv"
#         print(filename)
        df = pd.read_csv(filename)
        print(len(df["census_tract_number"].unique()))
        print(len(df["census_tract_number"]))
#         print(df.columns)
        df["Geoid"] = df.apply(lambda row: get_full_fips(row["census_tract_number"], row["county_name"]), axis=1)
#         df["Geoid"] = df.apply(lambda row: print(row["census_tract_number"], row["county_name"]), axis=1)

#         print(len(df["Geoid"].unique()))
#         print(len(df["Geoid"]))
        df["County"] = county
        new_frame = df[["Geoid", "County", "tract_to_msamd_income"]]
        year_dfs.append(new_frame)
        
    all_year = pd.concat(year_dfs, axis=0)
    all_year.to_csv(reference+year+"_all_year.csv", index=None)
    tables_dict[year] = all_year
    break
              

315
11808
4505.0 'Alameda County'
4003.0 'Alameda County'
4046.0 'Alameda County'
4004.0 'Alameda County'
4504.0 'Alameda County'
4403.02 'Alameda County'
4351.02 'Alameda County'
4006.0 'Alameda County'
4306.0 'Alameda County'
4007.0 'Alameda County'
4082.0 'Alameda County'
4382.01 'Alameda County'
4007.0 'Alameda County'
4035.0 'Alameda County'
4016.0 'Alameda County'
4035.0 'Alameda County'
4274.0 'Alameda County'
4380.0 'Alameda County'
4016.0 'Alameda County'
4044.0 'Alameda County'
4045.02 'Alameda County'
4081.0 'Alameda County'
4512.01 'Alameda County'
4417.0 'Alameda County'
4047.0 'Alameda County'
4068.0 'Alameda County'
4047.0 'Alameda County'
4081.0 'Alameda County'
4424.0 'Alameda County'
4378.0 'Alameda County'
4077.0 'Alameda County'
4081.0 'Alameda County'
4047.0 'Alameda County'
4081.0 'Alameda County'
4323.0 'Alameda County'
4005.0 'Alameda County'
4081.0 'Alameda County'
4081.0 'Alameda County'
4081.0 'Alameda County'
4102.0 'Alameda County'
4506.02 'Alameda County'


4307.0 'Alameda County'
4507.21 'Alameda County'
4376.0 'Alameda County'
4251.0 'Alameda County'
4306.0 'Alameda County'
4302.0 'Alameda County'
4307.0 'Alameda County'
4514.01 'Alameda County'
4416.02 'Alameda County'
4251.0 'Alameda County'
4307.0 'Alameda County'
4303.0 'Alameda County'
4307.0 'Alameda County'
4421.0 'Alameda County'
4506.04 'Alameda County'
4251.0 'Alameda County'
4307.0 'Alameda County'
4305.0 'Alameda County'
4307.0 'Alameda County'
4507.01 'Alameda County'
4359.0 'Alameda County'
4251.0 'Alameda County'
4307.0 'Alameda County'
4306.0 'Alameda County'
4312.0 'Alameda County'
4403.06 'Alameda County'
4305.0 'Alameda County'
4261.0 'Alameda County'
4307.0 'Alameda County'
4307.0 'Alameda County'
4321.0 'Alameda County'
4415.21 'Alameda County'
4381.0 'Alameda County'
4271.0 'Alameda County'
4309.0 'Alameda County'
4307.0 'Alameda County'
4321.0 'Alameda County'
4515.01 'Alameda County'
4420.0 'Alameda County'
4271.0 'Alameda County'
4311.0 'Alameda County'
4307.0 '

4423.0 'Alameda County'
4444.0 'Alameda County'
4100.0 'Alameda County'
4414.02 'Alameda County'
4431.01 'Alameda County'
4423.0 'Alameda County'
4423.0 'Alameda County'
4082.0 'Alameda County'
4283.01 'Alameda County'
4415.01 'Alameda County'
4431.01 'Alameda County'
4423.0 'Alameda County'
4423.0 'Alameda County'
4415.22 'Alameda County'
4351.02 'Alameda County'
4415.01 'Alameda County'
4431.02 'Alameda County'
4423.0 'Alameda County'
4424.0 'Alameda County'
4356.0 'Alameda County'
4358.0 'Alameda County'
4415.01 'Alameda County'
4431.02 'Alameda County'
4423.0 'Alameda County'
4424.0 'Alameda County'
4366.02 'Alameda County'
4366.01 'Alameda County'
4415.03 'Alameda County'
4431.02 'Alameda County'
4423.0 'Alameda County'
4424.0 'Alameda County'
4515.02 'Alameda County'
4372.0 'Alameda County'
4415.03 'Alameda County'
4431.02 'Alameda County'
4424.0 'Alameda County'
4426.0 'Alameda County'
4403.02 'Alameda County'
4372.0 'Alameda County'
4415.03 'Alameda County'
4431.02 'Alameda Cou

4376.0 'Alameda County'
4382.01 'Alameda County'
4340.0 'Alameda County'
4378.0 'Alameda County'
4371.0 'Alameda County'
4446.0 'Alameda County'
4378.0 'Alameda County'
4415.03 'Alameda County'
4351.02 'Alameda County'
4043.0 'Alameda County'
4371.0 'Alameda County'
4502.0 'Alameda County'
4379.0 'Alameda County'
4419.22 'Alameda County'
4356.0 'Alameda County'
4366.01 'Alameda County'
4371.0 'Alameda County'
4503.0 'Alameda County'
4381.0 'Alameda County'
4506.06 'Alameda County'
4364.01 'Alameda County'
4507.01 'Alameda County'
4371.0 'Alameda County'
4512.01 'Alameda County'
4403.02 'Alameda County'
4507.21 'Alameda County'
4364.01 'Alameda County'
4501.0 'Alameda County'
4371.0 'Alameda County'
4514.01 'Alameda County'
4403.05 'Alameda County'
4511.01 'Alameda County'
4370.0 'Alameda County'
4441.0 'Alameda County'
4371.0 'Alameda County'
4514.02 'Alameda County'
4403.08 'Alameda County'
4505.0 'Alameda County'
4372.0 'Alameda County'
4378.0 'Alameda County'
4371.0 'Alameda County'

4081.0 'Alameda County'
4303.0 'Alameda County'
4083.0 'Alameda County'
4301.0 'Alameda County'
4092.0 'Alameda County'
4507.03 'Alameda County'
4081.0 'Alameda County'
4304.0 'Alameda County'
4083.0 'Alameda County'
4302.0 'Alameda County'
4312.0 'Alameda County'
4507.21 'Alameda County'
4081.0 'Alameda County'
4305.0 'Alameda County'
4083.0 'Alameda County'
4302.0 'Alameda County'
4423.0 'Alameda County'
4507.21 'Alameda County'
4081.0 'Alameda County'
4306.0 'Alameda County'
4084.0 'Alameda County'
4302.0 'Alameda County'
4403.04 'Alameda County'
4507.22 'Alameda County'
4083.0 'Alameda County'
4306.0 'Alameda County'
4084.0 'Alameda County'
4302.0 'Alameda County'
4301.0 'Alameda County'
4507.22 'Alameda County'
4085.0 'Alameda County'
4307.0 'Alameda County'
4085.0 'Alameda County'
4303.0 'Alameda County'
4507.03 'Alameda County'
4507.22 'Alameda County'
4086.0 'Alameda County'
4307.0 'Alameda County'
4085.0 'Alameda County'
4304.0 'Alameda County'
4382.02 'Alameda County'
4507.22

...

607.0 'San Francisco County'
229.02 'San Francisco County'
607.0 'San Francisco County'
307.0 'San Francisco County'
353.0 'San Francisco County'
259.0 'San Francisco County'
607.0 'San Francisco County'
230.01 'San Francisco County'
610.0 'San Francisco County'
307.0 'San Francisco County'
354.0 'San Francisco County'
260.01 'San Francisco County'
607.0 'San Francisco County'
230.01 'San Francisco County'
610.0 'San Francisco County'
307.0 'San Francisco County'
354.0 'San Francisco County'
260.02 'San Francisco County'
607.0 'San Francisco County'
230.03 'San Francisco County'
610.0 'San Francisco County'
308.0 'San Francisco County'
426.0 'San Francisco County'
260.03 'San Francisco County'
607.0 'San Francisco County'
252.0 'San Francisco County'
610.0 'San Francisco County'
308.0 'San Francisco County'
428.0 'San Francisco County'
260.04 'San Francisco County'
607.0 'San Francisco County'
253.0 'San Francisco County'
177.0 'San Francisco County'
308.0 'San Francisco County'
452.0 

218.0 'San Francisco County'
215.0 'San Francisco County'
214.0 'San Francisco County'
607.0 'San Francisco County'
227.01 'San Francisco County'
216.0 'San Francisco County'
218.0 'San Francisco County'
230.01 'San Francisco County'
108.0 'San Francisco County'
313.0 'San Francisco County'
179.01 'San Francisco County'
217.0 'San Francisco County'
218.0 'San Francisco County'
256.0 'San Francisco County'
121.0 'San Francisco County'
260.02 'San Francisco County'
227.03 'San Francisco County'
218.0 'San Francisco County'
226.0 'San Francisco County'
257.0 'San Francisco County'
161.0 'San Francisco County'
134.0 'San Francisco County'
158.0 'San Francisco County'
230.01 'San Francisco County'
228.01 'San Francisco County'
260.01 'San Francisco County'
162.0 'San Francisco County'
607.0 'San Francisco County'
205.0 'San Francisco County'
256.0 'San Francisco County'
228.01 'San Francisco County'
263.01 'San Francisco County'
162.0 'San Francisco County'
176.01 'San Francisco County'
212

607.0 'San Francisco County'
176.02 'San Francisco County'
179.01 'San Francisco County'
311.0 'San Francisco County'
304.0 'San Francisco County'
228.01 'San Francisco County'
206.0 'San Francisco County'
201.0 'San Francisco County'
202.0 'San Francisco County'
262.0 'San Francisco County'
176.02 'San Francisco County'
179.01 'San Francisco County'
158.0 'San Francisco County'
426.0 'San Francisco County'
206.0 'San Francisco County'
201.0 'San Francisco County'
254.02 'San Francisco County'
331.0 'San Francisco County'
607.0 'San Francisco County'
170.0 'San Francisco County'
228.01 'San Francisco County'
607.0 'San Francisco County'
204.0 'San Francisco County'
126.0 'San Francisco County'
159.0 'San Francisco County'
211.0 'San Francisco County'
228.01 'San Francisco County'
227.01 'San Francisco County'
351.0 'San Francisco County'
402.0 'San Francisco County'
171.0 'San Francisco County'
354.0 'San Francisco County'
254.03 'San Francisco County'
256.0 'San Francisco County'
230.

1261.0 'Marin County'
1282.0 'Marin County'
1142.0 'Marin County'
1242.0 'Marin County'
1122.0 'Marin County'
1060.02 'Marin County'
1261.0 'Marin County'
1081.0 'Marin County'
1150.0 'Marin County'
1250.0 'Marin County'
1241.0 'Marin County'
1090.0 'Marin County'
1281.0 'Marin County'
1031.0 'Marin County'
1150.0 'Marin County'
1250.0 'Marin County'
1021.0 'Marin County'
1212.0 'Marin County'
1281.0 'Marin County'
1110.0 'Marin County'
1192.0 'Marin County'
1261.0 'Marin County'
1031.0 'Marin County'
1032.0 'Marin County'
1281.0 'Marin County'
1122.0 'Marin County'
1212.0 'Marin County'
1262.0 'Marin County'
1042.0 'Marin County'
1090.0 'Marin County'
1282.0 'Marin County'
1270.0 'Marin County'
1212.0 'Marin County'
1270.0 'Marin County'
1192.0 'Marin County'
1110.0 'Marin County'
1282.0 'Marin County'
1191.0 'Marin County'
1212.0 'Marin County'
1270.0 'Marin County'
1282.0 'Marin County'
1192.0 'Marin County'
1081.0 'Marin County'
1031.0 'Marin County'
1242.0 'Marin County'
1281.0 'M

2003.0 'Napa County'
2002.01 'Napa County'
2002.03 'Napa County'
2010.02 'Napa County'
2002.01 'Napa County'
2010.02 'Napa County'
2003.0 'Napa County'
2002.02 'Napa County'
2003.0 'Napa County'
2010.02 'Napa County'
2002.03 'Napa County'
2003.0 'Napa County'
2003.0 'Napa County'
2002.03 'Napa County'
2003.0 'Napa County'
2010.02 'Napa County'
2003.0 'Napa County'
2010.01 'Napa County'
2003.0 'Napa County'
2003.0 'Napa County'
2003.0 'Napa County'
2010.02 'Napa County'
2003.0 'Napa County'
2011.0 'Napa County'
2005.01 'Napa County'
2003.0 'Napa County'
2003.0 'Napa County'
2002.01 'Napa County'
2003.0 'Napa County'
2010.01 'Napa County'
2005.02 'Napa County'
2004.0 'Napa County'
2005.01 'Napa County'
2003.0 'Napa County'
2003.0 'Napa County'
2005.01 'Napa County'
2005.02 'Napa County'
2005.01 'Napa County'
2005.01 'Napa County'
2005.01 'Napa County'
2003.0 'Napa County'
2017.0 'Napa County'
2006.0 'Napa County'
2005.01 'Napa County'
2005.01 'Napa County'
2005.02 'Napa County'
2003.0 'N

In [133]:
# print(len(all_year["Geoid"].unique()))
# print(len(all_year["Geoid"]))
all_year["Geoid"]

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27    NaN
28    NaN
29    NaN
       ..
959   NaN
960   NaN
961   NaN
962   NaN
963   NaN
964   NaN
965   NaN
966   NaN
967   NaN
968   NaN
969   NaN
970   NaN
971   NaN
972   NaN
973   NaN
974   NaN
975   NaN
976   NaN
977   NaN
978   NaN
979   NaN
980   NaN
981   NaN
982   NaN
983   NaN
984   NaN
985   NaN
986   NaN
987   NaN
988   NaN
Name: Geoid, Length: 58072, dtype: float64

In [93]:
regression_data_filepath = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/regression/'


tracts_09 = pd.read_csv(regression_data_filepath+"all_vars_09.csv")
print(tracts_09.dtypes)
print(tables_dict["2009"].dtypes)
# merged_09 = pd.merge(tables_dict["2009"], tracts_09, on='Geoid')[["Geoid", "tract_to_msamd_income", '% Loans to MHI borrowers']]


Geoid                              int64
% Loans to MHI borrowers         float64
CRA                                int64
Pct Bachelor's degree            float64
% below poverty level            float64
% Single Family                  float64
Percent NH White alone           float64
Total number of housing units      int64
Median home value                float64
Median income                    float64
Total loans 2009                   int64
dtype: object
Geoid                     object
County                    object
tract_to_msamd_income    float64
dtype: object
